<a href="https://colab.research.google.com/github/phuongghj123/CS114.K21/blob/master/Case_study_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DANH SÁCH THÀNH VIÊN NHÓM:

*   18521060 Trịnh Hưng Long
*   18521062 Hà Văn Luân
*   18521274 Lỡ Đình Phương






**Mô tả bài toán**: Kiểm tra xem tiêu đề của một bài báo có phải là châm biếm hay không
*   Input: (headlines) tiêu đề của bài báo
*   Output: sarcasm hoặc là non-sarcasm

**Cách thu thập dataset**: Thu thập dataset từ trang web Kaggle (https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection) 

**Mô tả dataset**: gồm có 3 columns
*   is_sarcastic: mang giá trị 1 nếu bài viết đó là sarcasm, ngược lại mang giá trị 0
*   headline: tiêu đề bài báo
*   article_link: link bài báo

Upload file kaggle.json lên Google Colab

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"phuonglodinh","key":"68146f7710bca4bc9e8726f60553d465"}'}

Sau khi upload file kaggle.json ta tiến hành tạo thư mục .kaggle sau đó đưa file kaggle.json vào thư mục này

In [2]:
!ls -lha kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 25 13:53 kaggle.json


Download dataset từ Kaggle về Google Colab

In [3]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 54.6MB/s]


Giải nén dữ liệu vừa tải về sử dụng lệnh unzip

In [4]:
!unzip news-headlines-dataset-for-sarcasm-detection.zip

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


Tiến hành khai báo các thư viện cần thiết

In [5]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

Đọc dataset bằng cách sử dụng lệnh **pd.read_json** trong thư viện pandas, ở đây em sử dụng dataset từ file **Sarcasm_Headlines_Dataset_v2.json**

"lines = True" có nghĩa là cho phép đọc dữ liệu theo từng dòng

In [6]:
dataset = pd.read_json("Sarcasm_Headlines_Dataset_v2.json", lines = True)
dataset.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In ra kích thước dataset, kết quả in ra có dạng (x,y)
Trong đó:
*   x: số hàng
*   y: số cột



In [7]:
dataset.shape

(28619, 3)

Vì cột **article_link** không ảnh hưởng gì tới việc detect sarcasm nên ta tiến hành loại bỏ cột **article_link**

In [8]:
dataset = dataset.drop(['article_link'], axis = 1)

In [9]:
dataset.head()

,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...
2,0,eat your veggies: 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word 'strea...


Bước này ta tiến hành loại bỏ những đối tượng không cần thiết trong tiêu đề bài báo như:
*   Các ký tự đặc biệt
*   Các dấu câu
*   Các từ ngữ không cần thiết, thừa thãi (stop word) trong tiếng Anh



In [10]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words("english"))

dataset['headline'] = dataset['headline'].apply(lambda s: re.sub('[^a-zA-Z]', ' ', s))
dataset.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep totally nails why congress is falling...
2,0,eat your veggies deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word strea...


In [ ]:
X_data = []
Y_data = dataset['is_sarcastic']
def Pre_processing(data):
  for item in data:
    word_tokens = word_tokenize(item)
    text = [word for word in word_tokens if word not in stop_words]
    X_data.append(" ".join(text))

Pre_processing(dataset['headline'])


**Feature engineering**

TF-IDF (Term Frequency - Inverse Document Frequency):
*   TF: tần số xuất hiện của 1 từ trong 1 văn bản
*   IDF: tần số nghịch của 1 từ trong một tập các văn bản

Kỹ thuật TF-IDF dùng để tính toán tần suất xuất hiện của một từ trong văn bản, dựa trên đó để đánh giá mức độ quan trọng của từng từ trong văn bản

Tham khảo: https://codetudau.com/bag-of-words-tf-idf-xu-ly-ngon-ngu-tu-nhien/index.html

TfidfVectorizer dùng để chuyển đổi dữ liệu văn bản sang ma trận các features TF-IDF

Tham khảo: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [12]:
vectorizer = TfidfVectorizer(analyzer='word', max_features=6000)
vectorizer.fit(X_data)
X_data_tfidf = vectorizer.transform(X_data).toarray()

In [13]:
print(X_data_tfidf)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
print(X_data_tfidf.shape)

(28619, 6000)


Tiến hành chia train, test theo tỉ lệ 80:20

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X_data_tfidf, Y_data, test_size=0.2)

**Train data**

Lần lượt sử dụng 4 model cho bài toán phân lớp để train dữ liệu

In [28]:
modelnb = MultinomialNB()
modelnb.fit(X_train, Y_train)
print("Model MultinomialNB")
print("Train score: ", modelnb.score(X_train, Y_train))
print("Test score: ", modelnb.score(X_test, Y_test))

Model MultinomialNB
Train score:  0.8513649268399214
Test score:  0.7907058001397624


In [29]:
modelrdf = RandomForestClassifier(n_estimators=10)
modelrdf.fit(X_train, Y_train)
print("Model RandomForestClassifier")
print("Train score: ", modelrdf.score(X_train, Y_train))
print("Test score: ", modelrdf.score(X_test, Y_test))

Model RandomForestClassifier
Train score:  0.9829657130377811
Test score:  0.7496505939902166


In [30]:
modellr = LogisticRegression()
modellr.fit(X_train, Y_train)
print("Model LogisticRegression")
print("Train score: ", modellr.score(X_train, Y_train))
print("Test score: ", modellr.score(X_test, Y_test))

Model LogisticRegression
Train score:  0.8599694256387858
Test score:  0.7894828791055206


In [31]:
modelln = LinearSVC()
modelln.fit(X_train, Y_train)
print("Model LinearSVC")
print("Train score: ", modelln.score(X_train, Y_train))
print("Test score: ", modelln.score(X_test, Y_test))

Model LinearSVC
Train score:  0.900851714348111
Test score:  0.7830188679245284


Cách tính độ chính xác của model bằng **score** như trên chỉ cho ta biết phần trăm dữ liệu được phân loại đúng mà không chỉ ra được dữ liệu được phân loại như thế nào, nên ta sử dụng một ma trận được gọi là **confusion matrix**

**Confusion matrix** giúp ta có cái nhìn chi tiết hơn trong quá trình chọn lọc model dựa trên tập dữ liệu có sẵn

Bài toán này thuộc loại binary classification (có 2 lớp dữ liệu) nên ta sử dụng phương pháp **True/False Positive/Negative**, ta định nghĩa phương pháp này dựa trên confusion matrix

Tham khảo: https://machinelearningcoban.com/2017/08/31/evaluation/#-truefalse-positivenegative

Để đánh giá chất lượng của model ta sử dụng khái niệm F1-score, khái niệm này dựa trên 2 khái niệm khác là Precision và Recall

**Precision** được định nghĩa là tỉ lệ số điểm true positive trong số những điểm được phân loại là positive (TP + FP) 

    Precision = TP/(TP+FP)

**Recall** được định nghĩa là tỉ lệ số điểm true positive trong số những điểm thực sự là positive (TP + FN)

    Recall = TP/(TP+FN)

**F1-score** là harmonic mean của **precision** và **recall** 
    
    F1-score = 2(precision x recall)/(precision + recall)
    
F1 có giá trị nằm trong nửa khoảng (0;1]. F1 càng cao, bộ phân lớp càng tốt. F1 = 1 khi và chỉ khi cả precision và recall đều bằng 1

In [32]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
#Model Naive Bayes
y_pred_nb = modelnb.predict(X_test)
matrix_nb = confusion_matrix(Y_test, y_pred_nb)
print(matrix_nb)
print(f1_score(Y_test, y_pred_nb, average = 'macro')) #average = macro: trung bình cộng của các precision theo từng class

[[2417  559]
 [ 639 2109]]
0.7900980589979978


In [33]:
#Model RandomForestClassifier
y_pred_rdf = modelrdf.predict(X_test)
matrix_rdf = confusion_matrix(Y_test, y_pred_rdf)
print(matrix_rdf)
print(f1_score(Y_test, y_pred_rdf, average = 'macro')) #average = macro: trung bình cộng của các precision theo từng class

[[2387  589]
 [ 844 1904]]
0.7478552613705246


In [34]:
#Model LogisticRegression
y_pred_lr = modellr.predict(X_test)
matrix_lr = confusion_matrix(Y_test, y_pred_lr)
print(matrix_lr)
print(f1_score(Y_test, y_pred_lr, average = 'macro')) #average = macro: trung bình cộng của các precision theo từng class

[[2451  525]
 [ 680 2068]]
0.7885361309410501


In [35]:
#Model LinearSVC()
y_pred_ln = modelln.predict(X_test)
matrix_ln = confusion_matrix(Y_test, y_pred_ln)
print(matrix_ln)
print(f1_score(Y_test, y_pred_ln, average = 'macro')) #average = macro: trung bình cộng của các precision theo từng class

[[2380  596]
 [ 646 2102]]
0.7825058436268582


Ta thấy trong 4 model trên, model Naive Bayes tốt nhất, dựa trên F1-score = 0.79009

Vì vậy ta chọn model Naive Bayes cho bài toán này

Thu thập 2000 headline bao gồm sarcasm và non-sarcasm từ 2 trang web lần lượt là www.theonion.com và www.huffingtonpost.com

Trong đó, thu thập từ mỗi trang web 1000 headline tương ứng

Em sử dụng thư viện BeautifulSoup: là 1 package Python dùng để phân tích cú pháp các tài liệu HTML và XML.

In [37]:
import requests
from bs4 import BeautifulSoup

In [38]:
%cd "/content/drive/My Drive/"

/content/drive/My Drive


Thu thập dữ liệu từ trang www.theonion.com

Để tăng tính đa dạng em chia ra thu thập dữ liệu từ 2 đường link của trang The Onion là : "https://www.theonion.com/c/news-in-brief " và "https://www.theonion.com/c/news-in-photos ", ứng với mỗi đường link em lấy ra 500 dữ liệu

Trong lúc thu thập dữ liệu ta chú trọng vào 2 class là "sc-759qgu-0 cYlVdn cw4lnv-6 eXwNRE" và "sc-1out364-0 hMndXN js_link" chứa tiêu đề (headline) và đường link (article_link) của dữ liệu, ta dùng lệnh find_all để trích xuất ra (với href là đường link và hàm .get_text() để lấy tiêu đề của dữ liệu)

Vì số lượng cần thu thập là 500 mà trên trang "https://www.theonion.com/c/news-in-brief " và "https://www.theonion.com/c/news-in-photos " đều chỉ hiển thị khoảng 20 dữ liệu đầu tiên, muốn trích xuất tiếp thì phải chuyển sang trang tiếp theo (More stories), tìm đến các đuôi 'href' gắn sau url = "https://www.theonion.com/c/news-in-brief" để truy cập trang tiếp theo, thu thập thêm 20 dữ liệu nữa, thực hiện các công đoạn như vậy cho đến khi tìm đủ 500 dữ liệu ứng với mỗi đường link

In [ ]:
url = "https://www.theonion.com/c/news-in-brief"
url_1 = url
with open('/content/drive/My Drive/new_data_sarcasm.json', 'a') as file:

  for i in range(25):
  
    r1 = requests.get(url_1)
    coverpage = r1.content
    soup1 = BeautifulSoup(coverpage, 'lxml')
    soup2 = BeautifulSoup(coverpage, 'html5lib')
    coverpage_news = soup1.find_all('h2', class_= "sc-759qgu-0 cYlVdn cw4lnv-6 eXwNRE")
    coverpage_news_1 = soup2.find_all('a', class_='sc-1out364-0 hMndXN js_link')
  
    t = 12
    while(t <=50):
      a = coverpage_news_1[t]['href']
      b = coverpage_news_1[t].get_text()
   
    
      file.write('{"is_sarcastic": 1, "headline": "')
      file.write(b)
      file.write('", "article_link": "')
      file.write(a)
      file.write('"}\n')

      t += 2
    url_1 = url + coverpage_news_1[51]['href']
  file.close()

In [ ]:
url = "https://www.theonion.com/c/news-in-photos"
url_1 = url
with open('/content/drive/My Drive/new_data_sarcasm.json', 'a') as file:

  for i in range(25):
  
    r1 = requests.get(url_1)
    coverpage = r1.content
    soup1 = BeautifulSoup(coverpage, 'lxml')
    soup2 = BeautifulSoup(coverpage, 'html5lib')
    coverpage_news = soup1.find_all('h2', class_= "sc-759qgu-0 cYlVdn cw4lnv-6 eXwNRE")
    coverpage_news_1 = soup2.find_all('a', class_='sc-1out364-0 hMndXN js_link')
  
    t = 12
    while(t <=50):
      a = coverpage_news_1[t]['href']
      b = coverpage_news_1[t].get_text()
   
    
      file.write('{"is_sarcastic": 1, "headline": "')
      file.write(b)
      file.write('", "article_link": "')
      file.write(a)
      file.write('"}\n')

      t += 2
    url_1 = url + coverpage_news_1[51]['href']
  file.close()

Thu thập dữ liệu từ trang www.huffingtonpost.com

Đối với trang web này ta cần phải thêm dòng lệnh

Request(url1, headers={'User-Agent': 'XYZ/3.0'}) để được cấp phát quyền truy cập

Trong lúc thu thập dữ liệu ta chú trọng vào class 'card__link yr-card-headline' chứa tiêu đề (headline) và đường link (article_link) của dữ liệu

Dùng .text để lấy tiêu đề (headline),  tìm đến đuôi 'href' gắn sau 'https://www.huffingtonpost.co.uk' để trích xuất đường link (article link) của dữ liệu 

In [ ]:
from urllib.request import Request , urlopen 
from bs4 import BeautifulSoup
Data = []
with open('/content/drive/My Drive/new_data_sarcasm.json', 'a') as file:
  for i in range(1,35):
    url1 = 'https://www.huffingtonpost.co.uk/news/'+ str(i) +'/?guccounter=2'
    req = Request(url1, headers={'User-Agent': 'XYZ/3.0'})
    page = urlopen(req,timeout=10)
    soup = BeautifulSoup(page,"html.parser")
    headlines = soup.find_all('a',attrs={'class':'card__link yr-card-headline'})

    for i in range(len(headlines)):
      headline = headlines[i].text
      if headline not in Data:
        Data.append(headline)
        link = 'https://www.huffingtonpost.co.uk' + headlines[i]['href']

        file.write('{"is_sarcastic": 0, "headline": "')
        file.write(headline)
        file.write('", "article_link": "')
        file.write(link)
        file.write('"}\n')
  file.close()

Đọc file json gồm các mới được thu thập

In [39]:
dataset_new = pd.read_json("new_data_sarcasm.json", lines = True)
dataset_new.head()

,is_sarcastic,headline,article_link
0,1,Nancy Pelosi Calls Jamaal Bowman To Scold Him ...,https://politics.theonion.com/nancy-pelosi-cal...
1,1,"Gamers, We Just Spent 4 Days Trapped In A Roll...",https://ogn.theonion.com/gamers-we-just-spent-...
2,1,Defiant Florida Officials Announce They Will I...,https://www.theonion.com/defiant-florida-offic...
3,1,"Employee Leaves Performance Review With Clear,...",https://www.theonion.com/employee-leaves-perfo...
4,1,Officials Warn Defunding Police Could Lead To ...,https://www.theonion.com/officials-warn-defund...


In [40]:
dataset_new.shape

(2051, 3)

Loại bỏ cột article link vì không ảnh hưởng tới việc dự đoán sarcasm

In [41]:
dataset_new = dataset_new.drop(['article_link'], axis = 1)

In [42]:
dataset_new.head()

,is_sarcastic,headline
0,1,Nancy Pelosi Calls Jamaal Bowman To Scold Him ...
1,1,"Gamers, We Just Spent 4 Days Trapped In A Roll..."
2,1,Defiant Florida Officials Announce They Will I...
3,1,"Employee Leaves Performance Review With Clear,..."
4,1,Officials Warn Defunding Police Could Lead To ...


Xử lý văn bản: 
*   Các ký tự đặc biệt
*   Các dấu câu
*   Các từ ngữ không cần thiết, thừa thãi (stop word) trong tiếng Anh

In [ ]:
dataset_new['headline'] = dataset_new['headline'].apply(lambda s: re.sub('[^a-zA-Z]', ' ', s))
X_data_new = []
Y_data_new = dataset_new['is_sarcastic']
for item in dataset_new['headline']:
  word_tokens = word_tokenize(item)
  text = [word for word in word_tokens if word not in stop_words]
  X_data_new.append(" ".join(text))


Sử dụng kỹ thuật TF-IDF để chuyển dữ liệu sang ma trận

In [44]:
X_data_newtf = vectorizer.transform(X_data_new).toarray()

Sử dụng thuật toán naive bayes để predict bộ dữ liệu mới

In [45]:
y_pred_new = modelnb.predict(X_data_newtf)
print("Test score naive bayes: ", modelnb.score(X_data_newtf, Y_data_new))

Test score naive bayes:  0.7610921501706485


In [48]:
matrix_new = confusion_matrix(Y_data_new, y_pred_new)
print(matrix_new)

[[837 214]
 [276 724]]


In [49]:
print(f1_score(Y_data_new, y_pred_new, average='macro'))

0.7603647451781569


Nhận xét: Trên bộ dữ liệu mới, giá trị f1_score nhỏ hơn so với bộ dữ liệu cũ khi dùng model đã train trên tập dữ liệu cũ (0.7603 < 0.79009). Tức là độ chính xác giảm đi, có thể do các headline mới chứa những từ mới không có trong tập dữ liệu cũ

Đoạn chương trình thực hiện sarcasm detection cho bất kỳ headline được nhập vào

In [54]:
print("Nhap du lieu moi: ")
str1 = str(input())
str1 = re.sub(r'[^a-zA-Z]', ' ', str1)
word_tokens = word_tokenize(str1)
str1 = [word for word in word_tokens if word not in stop_words]
str1 = " ".join(str1)

Nhap du lieu moi: 
Pros And Cons Of D.C. Statehood


In [55]:
str1_tf = vectorizer.transform([str1]).toarray()

In [56]:
y_pred = modelnb.predict(str1_tf)

In [57]:
if (y_pred == 0):
  print("Not sarcasm")
else:
  print("Sarcasm")

Not sarcasm
